In [2]:
from urllib.request import urlopen
import pandas as pd
import numpy as np
import pdfplumber
import requests

import re

from bs4 import BeautifulSoup

import difflib

from io import BytesIO
import copy


In [19]:
#load in supreme court individual justice data
jcdf = pd.read_csv("/Users/eclin/Desktop/ML_Project/SCDB_2024_01_justiceCentered_Citation 2.csv")

/var/folders/mf/zhnnyffn3wv77d_kx1l8l8hw0000gn/T/ipykernel_13081/389403910.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  jcdf = pd.read_csv("/Users/eclin/Desktop/ML_Project/SCDB_2024_01_justiceCentered_Citation 2.csv")


In [35]:
#load in scraped transcripts
yrs = list(range(2000, 2024, 1))
oa = {}

for year in yrs: 
    path = f"/Users/eclin/Desktop/ML_Project/oral_arg/arguments{year}.json"
    oral_arg_df = pd.read_json(path)

    oa[year] = oral_arg_df

all_oral_args = pd.concat(oa)
all_oral_args = all_oral_args.drop_duplicates(subset = "caseId", keep = "first")
all_oral_args

#all_oral_args.to_csv("alloralargs.csv")
# all_oral_args.to_json("alloralargs_json.json")


year                                case_names  \
2000 0   2000                             TYLER V. CAIN   
     1   2000                    NEW HAMPSHIRE V. MAINE   
     2   2000       UNITED STATES V. UNITED FOODS, INC.   
     3   2000                   IDAHO V. UNITED STATES    
     4   2000  POLLARD V. E. I. DUPONT DE NEMOURS & CO.   
...       ...                                       ...   
2023 38  2023                        CULLEY V. MARSHALL   
     39  2023                   UNITED STATES V. RAHIMI   
     40  2023         RUDISILL V. MCDONOUGH, SEC. OF VA   
     41  2023                 PULSIFER V. UNITED STATES   
     42  2023       ALEXANDER V. SC CONFERENCE OF NAACP   

                                                scdb_match   dateArg  \
2000 0                   MELVIN TYLER v. BURL CAIN, WARDEN  04/16/01   
     1            STATE OF NEW HAMPSHIRE v. STATE OF MAINE  04/16/01   
     2   UNITED STATES AND DEPARTMENT OF AGRICULTURE v....  04/17/01   
     3                       IDAHO v. UNITED STATES et al.  04/23/01   
     4   SHARON B. POLLARD v. E. I. DU PONT DE NEMOURS ...  04/23/01   
...                                                    ...       ...   
2023 38                                 CULLEY v. MARSHALL  10/30/23   
     39                            UNITED STATES v. RAHIMI  11/07/23   
     40                              RUDISILL v. MCDONOUGH  11/08/23   
     41                          PULSIFER v. UNITED STATES  10/02/23   
     42  ALEXANDER v. SOUTH CAROLINA STATE CONFERENCE O...  10/11/23   

                                           oral_arg  \
2000 0           /pdfs/transcripts/2000/00-5961.pdf   
     1           /pdfs/transcripts/2000/130orig.pdf   
     2            /pdfs/transcripts/2000/00-276.pdf   
     3            /pdfs/transcripts/2000/00-189.pdf   
     4            /pdfs/transcripts/2000/00-763.pdf   
...                                             ...   
2023 38  /argument_transcripts/2023/22-585_d6od.pdf   
     39  /argument_transcripts/2023/22-915_6khn.pdf   
     40  /argument_transcripts/2023/22-888_olp1.pdf   
     41  /argument_transcripts/2023/22-340_hhdp.pdf   
     42  /argument_transcripts/2023/22-807_1chk.pdf   

                                                 full_link    caseId  \
2000 0   https://www.supremecourt.gov/pdfs/transcripts/...  2000-086   
     1   https://www.supremecourt.gov/pdfs/transcripts/...  2000-060   
     2   https://www.supremecourt.gov/pdfs/transcripts/...  2000-081   
     3   https://www.supremecourt.gov/pdfs/transcripts/...  2000-077   
     4   https://www.supremecourt.gov/pdfs/transcripts/...  2000-067   
...                                                    ...       ...   
2023 38  https://www.supremecourt.gov/oral_arguments/ar...  2023-007   
     39  https://www.supremecourt.gov/oral_arguments/ar...  2023-012   
     40  https://www.supremecourt.gov/oral_arguments/ar...  2023-013   
     41  https://www.supremecourt.gov/oral_arguments/ar...  2023-001   
     42  https://www.supremecourt.gov/oral_arguments/ar...  2023-006   

            docketId    caseIssuesId             voteId  ...  \
2000 0   2000-086-01  2000-086-01-01  2000-086-01-01-01  ...   
     1   2000-060-01  2000-060-01-01  2000-060-01-01-01  ...   
     2   2000-081-01  2000-081-01-01  2000-081-01-01-01  ...   
     3   2000-077-01  2000-077-01-01  2000-077-01-01-01  ...   
     4   2000-067-01  2000-067-01-01  2000-067-01-01-01  ...   
...              ...             ...                ...  ...   
2023 38  2023-007-01  2023-007-01-01  2023-007-01-01-01  ...   
     39  2023-012-01  2023-012-01-01  2023-012-01-01-01  ...   
     40  2023-013-01  2023-013-01-01  2023-013-01-01-01  ...   
     41  2023-001-01  2023-001-01-01  2023-001-01-01-01  ...   
     42  2023-006-01  2023-006-01-01  2023-006-01-01-01  ...   

                                              justice_text  \
2000 0   Well hear argument next in No. -, Melvin Tyler...   
     1   Well he

In [36]:
vdfs = {}
jcdf_tomerge = jcdf[["caseId", 'direction', 'firstAgreement','justice','justiceName','majority','opinion','secondAgreement', 'vote']]
for year in oa:
    tdf = oa[year]
    tdf["caseId"] = tdf["caseId"].astype(str)
    
    vdf = pd.merge(jcdf_tomerge, tdf, on = "caseId", how = "right")
    vdfs[year] = vdf

vdfs

{2000:        caseId  direction  firstAgreement  justice  justiceName  majority  \
 0    2000-086        1.0             NaN      102  WHRehnquist       2.0   
 1    2000-086        2.0           110.0      103    JPStevens       1.0   
 2    2000-086        1.0             NaN      104    SDOConnor       2.0   
 3    2000-086        1.0             NaN      105      AScalia       2.0   
 4    2000-086        1.0             NaN      106    AMKennedy       2.0   
 ..        ...        ...             ...      ...          ...       ...   
 453  2000-018        1.0           108.0      106    AMKennedy       1.0   
 454  2000-018        2.0             NaN      107     DHSouter       2.0   
 455  2000-018        1.0             NaN      108      CThomas       1.0   
 456  2000-018        2.0             NaN      109   RBGinsburg       2.0   
 457  2000-018        2.0             NaN      110     SGBreyer       2.0   
 
      opinion  secondAgreement  vote  year  ...  \
 0        1.0    

In [38]:
# Construst Matrix of Votes
alldfs = pd.concat(vdfs)
voteinfo = alldfs[["caseId", "justiceName", "majority"]]

#following model outlined in paper, if justice votes in majority, set vote = 1, else = 0
voteinfo["useVote"] = np.where(voteinfo["majority"] == 2, 1, 0)
voteinfo

/var/folders/mf/zhnnyffn3wv77d_kx1l8l8hw0000gn/T/ipykernel_13081/4242838689.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  voteinfo["useVote"] = np.where(voteinfo["majority"] == 2, 1, 0)


caseId  justiceName  majority  useVote
2000 0    2000-086  WHRehnquist       2.0        1
     1    2000-086    JPStevens       1.0        0
     2    2000-086    SDOConnor       2.0        1
     3    2000-086      AScalia       2.0        1
     4    2000-086    AMKennedy       2.0        1
...            ...          ...       ...      ...
2023 382  2023-006       EKagan       1.0        0
     383  2023-006    NMGorsuch       2.0        1
     384  2023-006  BMKavanaugh       2.0        1
     385  2023-006    ACBarrett       2.0        1
     386  2023-006    KBJackson       1.0        0

[10431 rows x 4 columns]

In [39]:
justice_vote_df = voteinfo.pivot_table(index = "caseId", columns = "justiceName", values = "useVote", aggfunc='first')
justice_vote_df

justice_vote_df.to_csv("justice_vote_matrix.csv")

In [40]:
justice_vote_df

justiceName,ACBarrett,AFortas,AMKennedy,AScalia,BMKavanaugh,BRWhite,CEWhittaker,CThomas,DHSouter,EKagan,...,SGBreyer,SMinton,SSotomayor,TCClark,TMarshall,WBRutledge,WEBurger,WHRehnquist,WJBrennan,WODouglas
caseId,,,,,,,,,,,,,,,,,,,,,
1947-031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0
1947-034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0
1949-114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0
1951-038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0
1954-017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-056,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-058,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-059,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
